# Agreement analysis for polarity

This notebook contains the functions to:
- analyze the manual review results. 
- create the gold standard based on the agreement
- obtain the observed and Kappa agreement

In [140]:
#####################
# Load Libraries    #
#####################
#install.packages("dplyr")
#install.packages("irr")
library("dplyr")
library("irr")
rm(list=ls())

ERROR: Error in library("ROCit"): there is no package called ‘ROCit’


In [43]:
###################################
# Function to transform the data  #
###################################
transformDataForAgreement <- function( input, startingColumn, finishingColum, output ){
    
    #remove the first row because is an example
    input <-  input[-1,]
    
    # Transform the dash into a 0 
    input[,c(startingColumn:finishingColum)] <- lapply(input[,c(startingColumn:finishingColum)], function(x) gsub("-", 0, x))
                                                       
    #Transform columns in numeric
    input[, c(startingColumn:finishingColum)] <- sapply(input[, c(startingColumn:finishingColum)], as.numeric)
    
    if( output == "dataframe"){
            return( input )
    }
    if( output == "vector" ){
        return( as.numeric( as.vector( t( input[,c(startingColumn:finishingColum)]))))
    }


}

In [44]:
#################################################
# Function to obtain the polarity gold standard #
#################################################
goldStandard <- function( dfSum, reviewers, startingColumn, finishingColumn ){
    
    dfSum$score <- NA
    dfSum$goldStandard <- NA
    
    for( i in 1:nrow( dfSum ) ){
        
        rowValues <- dfSum[i,startingColumn:finishingColumn]
        differentAnswers <- length( rowValues[ rowValues != 0])
        
        if( differentAnswers == 1){
            dfSum$score[i] <- 1
            dfSum$goldStandard[i] <- names(which.max(rowValues)) 
        }else if( differentAnswers == reviewers){
            dfSum$score[i] <- 0
            dfSum$goldStandard[i] <- "NO AGREEMENT"
        }else{
            dfSum$score[i] <- round( max(rowValues)/reviewers, 2)
            dfSum$goldStandard[i] <- names(which.max(rowValues))
        }
    }
    return( dfSum )
}

## Agreement analysis for polarity set 1

In [95]:
fileList <- list.files("/home/ec2-user/SageMaker/tweetsToReviewPolarity/", pattern = "set1_Results.csv")
fileList <- fileList[ fileList != "polarity_set1WithResults.csv" ]

rm(df1, df2, df3, df4, df5, a1, b1, c1, d1, e1)

for(i in 1:length(fileList)){
    newName <- paste0("df", i)
    filepath <- file.path("/home/ec2-user/SageMaker/tweetsToReviewPolarity/",paste0(fileList[i]))
    assign(newName, read.csv(filepath, header=TRUE))
}

df1 <- transformDataForAgreement( input = df1, startingColumn = 2, finishingColum = 6, output = "dataframe")
df1 <- df1[, c(1:6)]
df2 <- transformDataForAgreement( input = df2, startingColumn = 2, finishingColum = 6, output = "dataframe")
df2 <- df2[, c(1:6)]
df3 <- transformDataForAgreement( input = df3, startingColumn = 2, finishingColum = 6, output = "dataframe")
df3 <- df3[, c(1:6)]
df4 <- transformDataForAgreement( input = df4, startingColumn = 2, finishingColum = 6, output = "dataframe")
df4 <- df4[, c(1:6)]
df5 <- transformDataForAgreement( input = df5, startingColumn = 2, finishingColum = 6, output = "dataframe")
df5 <- df5[, c(1:6)]


a1 <- transformDataForAgreement( input = df1, startingColumn = 2, finishingColum = 6, output = "vector")
b1 <- transformDataForAgreement( input = df2, startingColumn = 2, finishingColum = 6, output = "vector")
c1 <- transformDataForAgreement( input = df3, startingColumn = 2, finishingColum = 6, output = "vector")
d1 <- transformDataForAgreement( input = df4, startingColumn = 2, finishingColum = 6, output = "vector")
e1 <- transformDataForAgreement( input = df5, startingColumn = 2, finishingColum = 6, output = "vector")

#############################################
# General Agreement between all the raters #
############################################
irr::kappam.fleiss(cbind(a1, b1, c1, d1, e1) )

dfSum <- cbind(df1[1], df1[-1] + df2[-1] + df3[-1] + df4[-1] + df5[-1])
dim(dfSum)


 Fleiss' Kappa for m Raters

 Subjects = 2435 
   Raters = 5 
    Kappa = 0.632 

        z = 98.7 
  p-value = 0 

[1] 488   6

### Agreement for each tweet between all the raters

In [96]:
kappaDf <- as.data.frame( df1[,1] )
kappaDf$kappa <- NA
kappaDf$pvalue <- NA
kappaDf$zscore <- NA

colnames( kappaDf )[1] <- "text"

for( i in 1:nrow( kappaDf ) ){
    kappaScore <- irr::kappam.fleiss(cbind(as.numeric(df1[i,2:6]), as.numeric(df2[i,2:6]), as.numeric(df3[i,2:6]),
                                     as.numeric(df4[i,2:6]), as.numeric(df5[i,2:6])))
    kappaDf$kappa[i]  <- kappaScore$value 
    kappaDf$pvalue[i] <- round(kappaScore$p.value,6)
    kappaDf$zscore[i] <- round(kappaScore$statistic, 3)
}

head(kappaDf)
print( paste0("Total agreement in ", nrow(kappaDf[ kappaDf$kappa ==1, ]), " out of the total ", nrow(kappaDf), " tweets."))

text,kappa,pvalue,zscore
"serious question my boyfriend has and now i’m curious; does my iud protect against god babies? like if there’s a market for immaculate conception, am i protected?",1.0,0.000000,7.071
has been proven not to work. for some it's psychological. they also use injection depo (contraception) on them too,0.5,0.000407,3.536
just because you have the blessing of being able to carry a child doesn’t mean you have the right to murder it. get a contraception implant if you don’t want to have a child. i’d much prefer you be barren instead of taking a life. idiot.....,1.0,0.000000,7.071
need response jennifer williams ward anyone taken depo provera? did you or do you like it? what to expect?,1.0,0.000000,7.071
lmfaooooo : let me see yo birth control patch bitch,0.5,0.000407,3.536
"« gotta birth control shot, ish hurt like hell» hell yeah",1.0,0.000000,7.071


[1] "Total agreement in 229 out of the total 488 tweets."


### Create the gold standard based on all the raters answers
- Each tweet has been reviewed independently by 5 different reviewers. We will include in the gold standard those tweets for which at least 3 reviewers agreed. 
- The goldStandard function will generate a data.frame where each row represent a tweet and it estimates how many raters classified the tweet as positive, negative, neutral, mixed or false positive. It will also show the Kappa score for the agreement (0 completely disagreement, 1 all agree) and the gold standard (the classification with the greates agreement)

In [97]:
dfSum <- cbind(df1[1], df1[-1] + df2[-1] + df3[-1] + df4[-1] + df5[-1])

goldStandardPolarity1 <- goldStandard( dfSum = dfSum, 
                                      reviewers = 5, 
                                      startingColumn = 2, 
                                      finishingColumn = 6 )
head( goldStandardPolarity1 )
summary(as.factor( goldStandardPolarity1$score))


,text,positive,negative,neutral,mixed,falsePositive,score,goldStandard
2,"serious question my boyfriend has and now i’m curious; does my iud protect against god babies? like if there’s a market for immaculate conception, am i protected?",0,0,5,0,0,1.0,neutral
3,has been proven not to work. for some it's psychological. they also use injection depo (contraception) on them too,0,1,4,0,0,0.8,neutral
4,just because you have the blessing of being able to carry a child doesn’t mean you have the right to murder it. get a contraception implant if you don’t want to have a child. i’d much prefer you be barren instead of taking a life. idiot.....,5,0,0,0,0,1.0,positive
5,need response jennifer williams ward anyone taken depo provera? did you or do you like it? what to expect?,0,0,5,0,0,1.0,neutral
6,lmfaooooo : let me see yo birth control patch bitch,0,0,4,1,0,0.8,neutral
7,"« gotta birth control shot, ish hurt like hell» hell yeah",0,5,0,0,0,1.0,negative


0.4 0.6 0.8   1 
 34 111 112 231

## Agreement analysis for polarity set 2

In [98]:
fileList <- list.files("/home/ec2-user/SageMaker/tweetsToReviewPolarity/", pattern = "set2_Results.csv")
fileList <- fileList[ fileList != "polarity_set2WithResults.csv" ]

rm(df1, df2, df3, df4, df5, a1, b1, c1, d1, e1)

for(i in 1:length(fileList)){
    newName <- paste0("df", i)
    filepath <- file.path("/home/ec2-user/SageMaker/tweetsToReviewPolarity/",paste0(fileList[i]))
    assign(newName, read.csv(filepath, header=TRUE))
}

df1 <- transformDataForAgreement( input = df1, startingColumn = 2, finishingColum = 6, output = "dataframe")
df2 <- transformDataForAgreement( input = df2, startingColumn = 2, finishingColum = 6, output = "dataframe")
df3 <- transformDataForAgreement( input = df3, startingColumn = 2, finishingColum = 6, output = "dataframe")
df4 <- transformDataForAgreement( input = df4, startingColumn = 2, finishingColum = 6, output = "dataframe")
df5 <- transformDataForAgreement( input = df5, startingColumn = 2, finishingColum = 6, output = "dataframe")

a1 <- transformDataForAgreement( input = df1, startingColumn = 2, finishingColum = 6, output = "vector")
b1 <- transformDataForAgreement( input = df2, startingColumn = 2, finishingColum = 6, output = "vector")
c1 <- transformDataForAgreement( input = df3, startingColumn = 2, finishingColum = 6, output = "vector")
d1 <- transformDataForAgreement( input = df4, startingColumn = 2, finishingColum = 6, output = "vector")
e1 <- transformDataForAgreement( input = df5, startingColumn = 2, finishingColum = 6, output = "vector")

#############################################
# General Agreement between all the raters #
############################################
irr::kappam.fleiss(cbind(a1, b1, c1, d1, e1) )
#irr::kappa2(cbind(a1, b1)) #agreement between pair of raters                   

dfSum <- cbind(df1[1], df1[-1] + df2[-1] + df3[-1] + df4[-1] + df5[-1])
dim(dfSum)



 Fleiss' Kappa for m Raters

 Subjects = 2420 
   Raters = 5 
    Kappa = 0.534 

        z = 83 
  p-value = 0 

[1] 485   6

### Agreement for each tweet between all the raters

In [99]:
kappaDf <- as.data.frame( df1[,1] )
kappaDf$kappa <- NA
kappaDf$pvalue <- NA
kappaDf$zscore <- NA

colnames( kappaDf )[1] <- "text"

for( i in 1:nrow( kappaDf ) ){
    kappaScore <- irr::kappam.fleiss(cbind(as.numeric(df1[i,2:6]), as.numeric(df2[i,2:6]), as.numeric(df3[i,2:6]),
                                     as.numeric(df4[i,2:6]), as.numeric(df5[i,2:6])))
    kappaDf$kappa[i]  <- kappaScore$value 
    kappaDf$pvalue[i] <- round(kappaScore$p.value,6)
    kappaDf$zscore[i] <- round(kappaScore$statistic, 3)
}


head(kappaDf)
print( paste0("Total agreement in ", nrow(kappaDf[ kappaDf$kappa ==1, ]), " out of the total ", nrow(kappaDf), " tweets."))

text,kappa,pvalue,zscore
i'm thinking about switching to the iud that they insert in your arm. it stays there for five years. so easy.,1.00,0.000000,7.071
imma blame it on the birth control shot,1.00,0.000000,7.071
tea tree oil!!!!! every night and morning i’ve been rubbing it all over my face w/ a make up wipe & it’s been working wonders! i have a bunch of acne from my iud! i’ve always had good skin!! i hate it!,0.50,0.000407,3.536
lmfaoo : what's that iud's : twitter: what's your fetish?,0.25,0.077100,1.768
since i have gotten my iud i’m not as emotional and don’t cry as much. which sucks because i miss having a good cry here and there,0.50,0.000407,3.536
"i don't need an over-compensator who doesn't know where iuds go to cure anything for me, but thanks for playing",0.25,0.077100,1.768


[1] "Total agreement in 172 out of the total 485 tweets."


### Create the gold standard based on all the raters answers
- Each tweet has been reviewed independently by 5 different reviewers. We will include in the gold standard those tweets for which at least 3 reviewers agreed. 
- The goldStandard function will generate a data.frame where each row represent a tweet and it estimates how many raters classified the tweet as positive, negative, neutral, mixed or false positive. It will also show the Kappa score for the agreement (0 completely disagreement, 1 all agree) and the gold standard (the classification with the greates agreement)

In [100]:
dfSum <- cbind(df1[1], df1[-1] + df2[-1] + df3[-1] + df4[-1] + df5[-1])

goldStandardPolarity2 <- goldStandard( dfSum = dfSum, 
                                      reviewers = 5, 
                                      startingColumn = 2, 
                                      finishingColumn = 6 )
head( goldStandardPolarity2 )
summary(as.factor( goldStandardPolarity2$score))

,text,positive,negative,neutral,mixed,falsePositive,score,goldStandard
2,i'm thinking about switching to the iud that they insert in your arm. it stays there for five years. so easy.,5,0,0,0,0,1.0,positive
3,imma blame it on the birth control shot,0,5,0,0,0,1.0,negative
4,tea tree oil!!!!! every night and morning i’ve been rubbing it all over my face w/ a make up wipe & it’s been working wonders! i have a bunch of acne from my iud! i’ve always had good skin!! i hate it!,0,4,0,1,0,0.8,negative
5,lmfaoo : what's that iud's : twitter: what's your fetish?,0,0,3,0,2,0.6,neutral
6,since i have gotten my iud i’m not as emotional and don’t cry as much. which sucks because i miss having a good cry here and there,0,1,0,4,0,0.8,mixed
7,"i don't need an over-compensator who doesn't know where iuds go to cure anything for me, but thanks for playing",0,0,3,2,0,0.6,neutral


0.4 0.6 0.8   1 
 50 146 117 172

## Create the gold standard file

In [118]:
gs1 <- goldStandardPolarity1[ goldStandardPolarity1$score > 0.4, ]
gs2 <- goldStandardPolarity2[ goldStandardPolarity2$score > 0.4, ]

print( paste0( "Total tweets included in the gold standard: ", nrow(finalGoldStandard)))

#Summary of the score and classification of the tweets in the gold standard
finalGoldStandard <- rbind( gs1, gs2 )
summary(as.factor(finalGoldStandard$score))
summary(as.factor(finalGoldStandard$goldStandard))

print( paste0( "Observed agreement: ", round((nrow(finalGoldStandard)/1000)*100, 2), "%"))

goldStandardFile <- finalGoldStandard[, c("text", "goldStandard")]
write.table( goldStandardFile, file = "/home/ec2-user/SageMaker/tweetsToReviewPolarity/finalGoldStandard.csv", 
             col.names = TRUE, row.names = FALSE, sep = "\t", quote = FALSE)

[1] "Total tweets included in the gold standard: 889"


0.6 0.8   1 
257 229 403

falsePositive         mixed      negative       neutral      positive 
           29            95           249           331           185

[1] "Observed agreement: 88.9%"


## Estimate the sensitivity and specificity
- We will compare the agreement values given by the reviewers with the results given by Amazon Comprehend to estimate the sensitivity and specificity

First we put together the automatic results and the ones included in the gold standard. 

In [122]:
# we select from the gold standard the three columns of interest, the text, the score and the gold standard
gs <- finalGoldStandard[, c("text", "score", "goldStandard")]
gs$text <- as.character(gs$text)

# we load the results for the same tweets from Amazon Comprehend
automaticResults2 <- read.csv("/home/ec2-user/SageMaker/tweetsToReviewPolarity/set2_AWScomprehend_corrected.csv", header = TRUE )
automaticResults2 <- automaticResults2[-1,-1]
automaticResults2$sentiments <- tolower( automaticResults2$sentiments )

automaticResults1 <- read.csv("/home/ec2-user/SageMaker/tweetsToReviewPolarity/set1_AWScomprehend_corrected.csv", header = TRUE )
automaticResults1 <- automaticResults1[-1,-1]
automaticResults1$sentiments <- tolower( automaticResults1$sentiments )

automaticResults <- rbind( automaticResults1, automaticResults2 )
automaticResults$tweets <- as.character(automaticResults$tweets)

head(automaticResults)

,tweets,sentiments,positive,negative,neutral,mixed
2,"serious question my boyfriend has and now i’m curious; does my iud protect against god babies? like if there’s a market for immaculate conception, am i protected?",neutral,0.1288393140,0.13338320,0.73731971,4.577281e-04
3,has been proven not to work. for some it's psychological. they also use injection depo (contraception) on them too,mixed,0.0021833186,0.02442839,0.01124731,9.621410e-01
4,just because you have the blessing of being able to carry a child doesn’t mean you have the right to murder it. get a contraception implant if you don’t want to have a child. i’d much prefer you be barren instead of taking a life. idiot.....,negative,0.0007591724,0.97708726,0.02213454,1.893010e-05
5,need response jennifer williams ward anyone taken depo provera? did you or do you like it? what to expect?,neutral,0.0015469046,0.01033810,0.98811311,1.891628e-06
6,lmfaooooo : let me see yo birth control patch bitch,negative,0.0102170259,0.90682310,0.08295346,6.405082e-06
7,"« gotta birth control shot, ish hurt like hell» hell yeah",negative,0.0699670836,0.47252899,0.44854924,8.954676e-03


Then we compare the results between both, creating as an output two different datasets, one with the tweets in agreement between the reviewers and Amazon Comprehend, and the other one with the results in disagreement. 

**Note that Amazon Comprehend gives a confident score for each output, that we are not considering when looking for the agreement**

In [126]:
agreeResults <- as.data.frame( matrix( ncol = 4) )
nonAgreeResults <- as.data.frame( matrix( ncol = 5) )

for( i in 1:nrow( gs ) ){
    #print(i)
    selection <- automaticResults[ automaticResults$tweets == gs$text[i], ]
    
    if(nrow(selection) == 0){
        #print( paste0(i, " row not found"))
    }
    else{
        if( selection$sentiments == gs$goldStandard[i] ){
            newRow <- c( as.character(gs$text[i]), gs$goldStandard[i], gs$score[i], max( selection[,c("positive","negative", "neutral")]))
            agreeResults <- rbind( agreeResults, newRow)
        }
        if( selection$sentiments != gs$goldStandard[i] ){
            newRow <- c( as.character(gs$text[i]), gs$goldStandard[i], gs$score[i], selection$sentiments, selection[, which(colnames(selection)== selection$sentiments)] )
            nonAgreeResults <- rbind( nonAgreeResults, newRow)
        } 
    }
}

agreeResults <- agreeResults[-1, ]
colnames(agreeResults) <- c("tweet", "commonAnswer", "manualScore", "awsScore")

nonAgreeResults <- nonAgreeResults[-1,]
colnames(nonAgreeResults) <- c("tweet", "manualAnswer", "manualScore", "awsAnswer", "awsScore")

print( paste0( "There are ", nrow(agreeResults), " tweets in agreement between reviewers and Amazon Comprehend"))
print( paste0( "There are ", nrow(nonAgreeResults), " tweets in disagreement between reviewers and Amazon Comprehend"))

[1] "There are 418 tweets in agreement between reviewers and Amazon Comprehend"
[1] "There are 397 tweets in disagreement between reviewers and Amazon Comprehend"


#### Estimate the scpecificity and sensitivity for each category

In [127]:
###################################################################
## Function to estimate sensitivity and specificity by category ##
##################################################################
#sensitivity or recall Se = TP/(TP+FN) 
#proportion of positive cases correctly identified
#specificity or precision Sp = TP/(TP+FP) ? 

sensitivitySepcificity <- function( category, awsScore = 0 ){
    truepositive <- nrow( agreeResults[ agreeResults$commonAnswer == category & agreeResults$awsScore >= awsScore, ] )
    truenegative <- nrow( agreeResults[ agreeResults$commonAnswer != category  & agreeResults$awsScore >= awsScore, ] )
    falsepositive <- nrow( nonAgreeResults[ nonAgreeResults$manualAnswer != category & nonAgreeResults$awsAnswer == category & nonAgreeResults$awsScore >= awsScore, ])
    falsenegative <- nrow( nonAgreeResults[ nonAgreeResults$manualAnswer == category & nonAgreeResults$awsAnswer != category & nonAgreeResults$awsScore >= awsScore , ])

    print( paste0( "True positive: ", truepositive))
    print( paste0( "True negative: ", truenegative))
    print( paste0( "False positive: ", falsepositive))
    print( paste0( "False negative: ", falsenegative))

    sensitivity = truepositive / (truepositive + falsenegative)
    print( paste0( "Sensitivity: ", sensitivity ))
    specificity = truenegative / (truenegative + falsepositive)
    print( paste0( "Specificity: ", specificity ))
}

We will estimate the general sensitivity and specificity without filtering by the Amazon Comprehend Score for each category:
- positive
- negative 
- neutral
- mixed

In [132]:
print( "Positive")
sensitivitySepcificity( category = "positive", awsScore = 0 )
print("###########")

print( "Negative")
sensitivitySepcificity( category = "negative", awsScore = 0 )
print("###########")

print( "Neutral")
sensitivitySepcificity( category = "neutral", awsScore = 0 )
print("###########")

print( "Mixed")
sensitivitySepcificity( category = "mixed", awsScore = 0 )
print("###########")

[1] "Positive"
[1] "True positive: 62"
[1] "True negative: 356"
[1] "False positive: 65"
[1] "False negative: 106"
[1] "Sensitivity: 0.369047619047619"
[1] "Specificity: 0.845605700712589"
[1] "###########"
[1] "Negative"
[1] "True positive: 153"
[1] "True negative: 265"
[1] "False positive: 178"
[1] "False negative: 77"
[1] "Sensitivity: 0.665217391304348"
[1] "Specificity: 0.598194130925508"
[1] "###########"
[1] "Neutral"
[1] "True positive: 190"
[1] "True negative: 228"
[1] "False positive: 135"
[1] "False negative: 108"
[1] "Sensitivity: 0.63758389261745"
[1] "Specificity: 0.628099173553719"
[1] "###########"
[1] "Mixed"
[1] "True positive: 13"
[1] "True negative: 405"
[1] "False positive: 19"
[1] "False negative: 80"
[1] "Sensitivity: 0.139784946236559"
[1] "Specificity: 0.955188679245283"
[1] "###########"


In [ ]:
print( "Positive")
sensitivitySepcificity( category = "positive", awsScore = 80 )
print("###########")

print( "Negative")
sensitivitySepcificity( category = "negative", awsScore = 80 )
print("###########")

print( "Neutral")
sensitivitySepcificity( category = "neutral", awsScore = 80 )
print("###########")

print( "Mixed")
sensitivitySepcificity( category = "mixed", awsScore = 80 )
print("###########")

We will estimate the general sensitivity and specificity filtering the Amazon Comprehend Score at 0.95 confident score for each category:
- positive
- negative 
- neutral
- mixed

In [133]:
print( "Positive")
sensitivitySepcificity( category = "positive", awsScore = 0.95 )
print("###########")

print( "Negative")
sensitivitySepcificity( category = "negative", awsScore = 0.95 )
print("###########")

print( "Neutral")
sensitivitySepcificity( category = "neutral", awsScore = 0.95 )
print("###########")

print( "Mixed")
sensitivitySepcificity( category = "mixed", awsScore = 0.95 )
print("###########")

[1] "Positive"
[1] "True positive: 22"
[1] "True negative: 98"
[1] "False positive: 9"
[1] "False negative: 11"
[1] "Sensitivity: 0.666666666666667"
[1] "Specificity: 0.91588785046729"
[1] "###########"
[1] "Negative"
[1] "True positive: 58"
[1] "True negative: 62"
[1] "False positive: 25"
[1] "False negative: 12"
[1] "Sensitivity: 0.828571428571429"
[1] "Specificity: 0.71264367816092"
[1] "###########"
[1] "Neutral"
[1] "True positive: 40"
[1] "True negative: 80"
[1] "False positive: 11"
[1] "False negative: 14"
[1] "Sensitivity: 0.740740740740741"
[1] "Specificity: 0.879120879120879"
[1] "###########"
[1] "Mixed"
[1] "True positive: 0"
[1] "True negative: 120"
[1] "False positive: 3"
[1] "False negative: 9"
[1] "Sensitivity: 0"
[1] "Specificity: 0.975609756097561"
[1] "###########"


In [129]:
agree <- summary( as.numeric(agreeResults$awsScore) )
notagree <- summary( as.numeric(nonAgreeResults$awsScore) )

print( paste0("Total tweets agreement between manual reviewers: ", nrow( gs) ))
print( paste0("Total tweets agreement between manual reviewers and aws: ", nrow( agreeResults) ))

[1] "Total tweets agreement between manual reviewers: 889"
[1] "Total tweets agreement between manual reviewers and aws: 418"


In [134]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Amazon Linux AMI 2018.03

Matrix products: default
BLAS/LAPACK: /home/ec2-user/anaconda3/envs/R/lib/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] irr_0.84.1       lpSolve_5.6.13.3 dplyr_0.8.3     

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1       digest_0.6.18    crayon_1.3.4     assertthat_0.2.1
 [5] IRdisplay_0.7.0  repr_0.19.2      R6_2.4.0         jsonlite_1.6    
 [9] magrittr_1.5     evaluate_0.13    pillar_1.3.1     rl